In [1]:
# import dependencies
import os
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import numpy as np
import time
import datetime
import json
import requests as req
from citipy import citipy
from itertools import cycle, islice

In [2]:
# get dataset, created a smaller random sample of dataset called 'flights_smaller.csv that has 100,000 rows
flights_dataset_path = os.path.join('Data', 'flights_smaller.csv')
airlines_dataset_path = os.path.join('Data', 'airlines.csv')
dataset = pd.read_csv(flights_dataset_path)
airport_data = json.load(open('Data/airports.json', 'r'))
airlines = pd.read_csv(airlines_dataset_path)

In [4]:
# convert to dataframe
flight_df = pd.DataFrame(dataset)
airlines_df = pd.DataFrame(airlines)
#Subset the data to airports with the most traffic(ATL, LAX, ORD, DFW, JFK, DEN, SFO, LAS, CLT, SEA airports)
flight_df = flight_df.loc[(flight_df['ORIGIN_AIRPORT'] == 'ATL') | (flight_df['ORIGIN_AIRPORT'] == 'LAX') | 
                           (flight_df['ORIGIN_AIRPORT'] == 'ORD') | (flight_df['ORIGIN_AIRPORT'] == 'DFW') |
                           (flight_df['ORIGIN_AIRPORT'] == 'JFK') | (flight_df['ORIGIN_AIRPORT'] == 'DEN') |
                           (flight_df['ORIGIN_AIRPORT'] == 'SFO') | (flight_df['ORIGIN_AIRPORT'] == 'LAS') |
                           (flight_df['ORIGIN_AIRPORT'] == 'CLT') | (flight_df['ORIGIN_AIRPORT'] == 'SEA')]

Series([], Name: MONTH, dtype: int64)

In [ ]:
# add delayed column to show if flight was delayed (1) or not (0)
flight_df['DELAYED'] = ''
flight_df['AIRLINE_NAME'] = ''
# add column for the date
flight_df['DATE'] = ''
# create column for unix timestamp
flight_df['UNIX_TIME'] = ''
# columns for lat and lon
flight_df['ORIGIN_LAT'] = ''
flight_df['ORIGIN_LON'] = ''
flight_df['ORIGIN_CITY'] = ''
flight_df['DEST_LAT'] = ''
flight_df['DEST_LON'] = ''
flight_df['DEST_CITY'] = ''
flight_df.head()

In [ ]:
# set Delayed column if flight was delayed
for index, row in flight_df.head(30).iterrows():
    # TODO: check if other columns are also null
    if(pd.isnull(row['AIR_SYSTEM_DELAY']) == False):
        flight_df.set_value(index, 'DELAYED', 1)
    
    date = str(row['MONTH']) + '/' + str(row['DAY']) + '/' + str(row['YEAR'])
    
    flight_df.set_value(index, 'DATE', date)

In [ ]:
for index, row in flight_df.head(30).iterrows():
    unix_time = time.mktime(datetime.datetime.strptime(row['DATE'], "%m/%d/%Y").timetuple())
    
    flight_df.set_value(index, 'UNIX_TIME', int(unix_time))

In [ ]:
flight_df.head()

In [ ]:
# get average stats for each airline
avg_airline_stats_df = flight_df.groupby('AIRLINE').mean()

In [ ]:
# remove unecessary columns that don't need averaged
avg_airline_stats_df.drop(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_TIME', 'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'DIVERTED', 'CANCELLED'], axis=1, inplace=True)

In [ ]:
# sort by the average arrival delay from longest avg delay to shortest avg delay
avg_airline_stats_df.reset_index(level=0, inplace=True)
avg_airline_stats_df.sort_values('ARRIVAL_DELAY', ascending=False)

In [ ]:
#bar chart of delays by airline.  TODO: creat a color for each and 
avg_airline_stats_df.plot(x = 'AIRLINE', y = 'ARRIVAL_DELAY', kind='bar', figsize = (12,4), legend=False)
plt.title("Average delay by airline in 2015")
plt.xlabel("Airline")
plt.ylabel("Average Delay")
plt.show()

In [ ]:
# get average stats for each month
avg_month_stats_df = flight_df.groupby('MONTH').mean()
avg_month_stats_df.drop(['YEAR', 'DAY', 'DAY_OF_WEEK', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_TIME', 'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'DIVERTED', 'CANCELLED'], axis=1, inplace=True)
avg_month_stats_df.reset_index(level=0, inplace=True)
avg_month_stats_df.sort_values('MONTH', ascending=True)

In [ ]:
#bar chart of delays by month.  TODO: creat a color for each and figure out why october isnt in there
avg_month_stats_df.plot(x = 'MONTH', y = 'ARRIVAL_DELAY', kind='bar', figsize = (12,4), legend=False)
plt.title("Average delay by Month in 2015")
plt.xlabel("Month")
plt.ylabel("Average Delay")
plt.show()

In [ ]:
# get average stats for each day
avg_day_stats_df = flight_df.groupby('DAY_OF_WEEK').mean()
avg_day_stats_df.drop(['YEAR', 'DAY', 'MONTH', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_TIME', 'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'DIVERTED', 'CANCELLED'], axis=1, inplace=True)
avg_day_stats_df.reset_index(level=0, inplace=True)
avg_day_stats_df.sort_values('DAY_OF_WEEK', ascending=False)

In [ ]:
#bar chart of delays by month.  TODO: creat a color for each and 
avg_day_stats_df.plot(x = 'DAY_OF_WEEK', y = 'ARRIVAL_DELAY', kind='bar', figsize = (12,4), legend=False)
plt.title("Average delay by Day in 2015")
plt.xlabel("Day")
plt.ylabel("Average Delay")
plt.show()

In [ ]:
avg_airport_stats_df = flight_df.groupby('ORIGIN_AIRPORT').mean()
avg_airport_stats_df.drop(['YEAR', 'DAY', 'MONTH', 'FLIGHT_NUMBER', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'WHEELS_OFF', 'SCHEDULED_TIME', 'WHEELS_ON', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'DIVERTED', 'CANCELLED'], axis=1, inplace=True)
avg_airport_stats_df.reset_index(level=0, inplace=True)
avg_airport_stats_df.sort_values('ORIGIN_AIRPORT', ascending=False)

In [ ]:
#bar chart of delays by airport.  TODO: creat a color for each and 
avg_airport_stats_df.plot(x = 'ORIGIN_AIRPORT', y = 'ARRIVAL_DELAY', kind='bar', figsize = (12,4), legend=False)
plt.title("Average delay by airport in 2015")
plt.xlabel("Airport")
plt.ylabel("Average Delay")
plt.show()

In [ ]:
# TODO: loop through all rows and get lat/lon
# TODO: use this info to get the weather for that airport on that day
# USE JSON TO GET CITY NAME, DONT NEED LAT & LON
#for index, row in flight_df.iterrows():
#    for key in airport_data:
#        if key['code'] == row['ORIGIN_AIRPORT']:
#            #city.append(key['city'])
#            flight_df.loc[index, 'origin_city'] = key['city']
#            break


In [ ]:
airport_city_code = {}
for key in airport_data:
    airport_city_code[key['code']] = key['city']

airport_city_code['JFK']

In [ ]:
def get_city_from(airport_code):
    return airport_city_code.get(airport_code, "N/A")

flight_df['ORIGIN_AIRPORT'].head().apply(get_city_from)

In [ ]:
flight_df['origin_city'] = flight_df['ORIGIN_AIRPORT'].apply(get_city_from)

In [ ]:
flight_df.head()

In [ ]:
for index, row in flight_df.head(29).iterrows():
    try:
        origin_city = citipy.nearest_city(float(row['origin_lat']), float(row['origin_lon']))
        dest_city = citipy.nearest_city(float(row['dest_lat']), float(row['dest_lon']))
        flight_df.set_value(index, 'origin_city', origin_city.city_name)
        flight_df.set_value(index, 'dest_city', dest_city.city_name)
    except:
        print('error at row ' + str(index))

In [ ]:
flight_df.head()

In [ ]:
cities.append(city.city_name)

In [ ]:
weather = req.get(weather_url).json()

In [ ]:
weather